모듈 ∙ 라이브러리 import

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
tf.config.experimental.set_visible_devices([], 'GPU')
# tf.config.experimental.list_physical_devices('GPU')

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt

데이터 불러오기 ∙ 전처리

In [2]:
def filter_df(df): #데이터프레임 필터링
    # 첫 번째 열에서 같은 값을 가진 행의 수를 계산합니다.
    row_counts = df['0'].value_counts()

    # 가장 많은 행의 수를 찾습니다.
    max_row_count = row_counts.max()

    # 가장 많은 행의 수에 해당하는 행만 분류합니다.
    filtered = pd.DataFrame(df[df['0'].isin(row_counts[row_counts == max_row_count].index)])

    return filtered

# 데이터 로드   
stock_df = pd.read_csv('/Users/moon/Desktop/Moon SeungHoo/Stock_Machine_Learning/StockData_3%.csv',low_memory=False)

#데이터 필터링
filter_stock = filter_df(stock_df)
filter_label = filter_stock['25']

# 불필요한 데이터 삭제
filter_stock = filter_stock.drop({'0','1','7','25'},axis=1) #날자, 상승율, 5%이상 상승여부 삭제 

데이터 전처리

In [3]:
scaler = MinMaxScaler()
stock_label = scaler.fit_transform(filter_label.values.reshape(-1, 1))

count_zeros = np.sum(stock_label == 0)
count_ones = np.sum(stock_label == 1)
total_samples = len(stock_label)

ratio_zeros = (count_zeros / total_samples).round(2)
ratio_ones = (count_ones / total_samples).round(2)

print("Ratio of zeros (0s):", ratio_zeros)
print("Ratio of ones (1s):", ratio_ones)

Ratio of zeros (0s): 0.97
Ratio of ones (1s): 0.03


데이터 분할

In [4]:
# 훈련 및 테스트 데이터로 분할
X_train, X_test, y_train, y_test = train_test_split(filter_stock, stock_label, test_size=0.2, random_state=42)

under_sampler = RandomUnderSampler(sampling_strategy='auto' , random_state=42)
X_resampled, y_resampled = under_sampler.fit_resample(X_train, y_train)
X_test_resampled, y_test_resampled = under_sampler.fit_resample(X_test, y_test)

# smote = SMOTE(sampling_strategy=0.7 ,random_state=42)
# X_train_resampled, y_train_resampled = smote.fit_resample(X_train,y_train)
# X_test_resampled, y_test_resampled = smote.fit_resample(X_test,y_test)

count_zeros = np.sum(y_resampled == 0)
count_ones = np.sum(y_resampled == 1)
total_samples = len(y_resampled)

ratio_zeros = (count_zeros / total_samples).round(2)
ratio_ones = (count_ones / total_samples).round(2)

print("Ratio of zeros (0s):", ratio_zeros)
print("Ratio of ones (1s):", ratio_ones)

Ratio of zeros (0s): 0.5
Ratio of ones (1s): 0.5


GRU 모델 정의

In [5]:
# GRU 모델 정의
tf.random.set_seed(42)
model = tf.keras.Sequential()
model.add(tf.keras.layers.GRU(64, input_shape=(X_resampled.shape[1],1),kernel_initializer='glorot_uniform', recurrent_dropout=0.0 ,return_sequences=True))
model.add(tf.keras.layers.GRU(64, return_sequences=True))
model.add(tf.keras.layers.GRU(64, return_sequences=True))
model.add(tf.keras.layers.GRU(64, return_sequences=True))
model.add(tf.keras.layers.GRU(64, return_sequences=True))
model.add(tf.keras.layers.GRU(64, return_sequences=True))
model.add(tf.keras.layers.GRU(64, return_sequences=True))
model.add(tf.keras.layers.GRU(64, return_sequences=False))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 조기 종료 콜백 정의
early_stopping = EarlyStopping(monitor='accuracy', patience=30, restore_best_weights=True)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru (GRU)                   (None, 27, 64)            12864     
                                                                 
 gru_1 (GRU)                 (None, 27, 64)            24960     
                                                                 
 gru_2 (GRU)                 (None, 27, 64)            24960     
                                                                 
 gru_3 (GRU)                 (None, 27, 64)            24960     
                                                                 
 gru_4 (GRU)                 (None, 27, 64)            24960     
                                                                 
 gru_5 (GRU)                 (None, 27, 64)            24960     
                                                                 
 gru_6 (GRU)                 (None, 27, 64)            2

모델 훈련

In [6]:
# 모델 훈련
model.fit(X_resampled,y_resampled,epochs=500,
validation_data=(X_test_resampled, y_test_resampled),callbacks=[early_stopping])
 
# 모델 저장
model.save("GRU_Model_8L_64_3%.h5")

Epoch 1/500
1713/1713 [==============================] - 227s 130ms/step - loss: 0.4284 - accuracy: 0.8119 - val_loss: 0.3505 - val_accuracy: 0.8475
Epoch 2/500
1713/1713 [==============================] - 224s 131ms/step - loss: 0.3432 - accuracy: 0.8554 - val_loss: 0.3545 - val_accuracy: 0.8485
Epoch 3/500
1713/1713 [==============================] - 223s 130ms/step - loss: 0.3217 - accuracy: 0.8647 - val_loss: 0.3176 - val_accuracy: 0.8769
Epoch 4/500
1713/1713 [==============================] - 222s 130ms/step - loss: 0.3101 - accuracy: 0.8697 - val_loss: 0.2733 - val_accuracy: 0.8887
Epoch 5/500
1713/1713 [==============================] - 236s 138ms/step - loss: 0.3037 - accuracy: 0.8719 - val_loss: 0.3018 - val_accuracy: 0.8849
Epoch 6/500
1713/1713 [==============================] - 227s 132ms/step - loss: 0.2955 - accuracy: 0.8767 - val_loss: 0.2695 - val_accuracy: 0.8878
Epoch 7/500
1713/1713 [==============================] - 226s 132ms/step - loss: 0.2916 - accuracy: 0.8781

/Users/moon/anaconda3/envs/ML/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


모델 평가

In [7]:
# 모델 로드
# loaded_model = tf.keras.models.load_model("GRU_Model_8L_64_10%.h5")

# 모델을 사용하여 주가 상승 여부 예측
test = model.predict(X_test_resampled)

13183/13183 [==============================] - 141s 11ms/step


In [8]:
y_pred_binary = (test > 0.5).astype(int)

accuracy = accuracy_score(y_test_resampled, y_pred_binary)
conf_matrix = confusion_matrix(y_test_resampled, y_pred_binary)
classification_rep = classification_report(y_test_resampled, y_pred_binary)

print(f"Accuracy: {accuracy}")
print("Confusion Matrix:")
print(conf_matrix)
print("Classification Report:")
print(classification_rep)

Accuracy: 0.8781903734695571
Confusion Matrix:
[[214048  34102]
 [ 17284 156421]]
Classification Report:
              precision    recall  f1-score   support

         0.0       0.93      0.86      0.89    248150
         1.0       0.82      0.90      0.86    173705

    accuracy                           0.88    421855
   macro avg       0.87      0.88      0.88    421855
weighted avg       0.88      0.88      0.88    421855

